In [3]:
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ae/d9/3741b344f57484b423cd22194025a8489992ad9962196a62721ef9980045/pandas-2.1.4-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.1 from https://files.pythonhosted.org/packages/a3/fb/52b62131e21b24ee297e4e95ed41eba29647dad0e0051a92bb66b43c70ff/tzdata-2023.4-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.5 MB 991.0 kB/s eta 0:00:11
   ---------------------------------------- 0.1/10.5 MB 1.1 MB/s eta 0:00:10
    --------------------------------------- 0.2/10.5 MB 2.0 MB/s eta 0:00:06
   - -------------------------------------- 0.3/10.5 MB 1.7 MB/s eta 


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:

# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle

In [5]:
path = "D:/workspace/OpenCV/"
# hiển thị tất cả thư mục hoặc file
os.listdir(path)

['.idea',
 'dataset',
 'FaceDetect.py',
 'FaceRecognition.py',
 'FaceTraining.py',
 'haarcascade_frontalface_default.xml',
 'trainer']

In [6]:
# face detection model
faceDetectionModel = './models/res10_300x300_ssd_iter_140000_fp16.caffemodel'

# mô tả kiến trúc
faceDetectionProto = './models/deploy.prototxt.txt'

# mô tả khuôn mặt
faceDescriptor = './models/openface.nn4.small2.v1.t7'

# sử dụng opencv dnn đọc mô hình nhận diện khuôn mặt
detectorModel = cv2.dnn.readNetFromCaffe(faceDetectionProto, faceDetectionModel)

# đọc mô hình mô tả khuôn mặt từ PyTorch
descriptorModel = cv2.dnn.readNetFromTorch(faceDescriptor)

In [7]:
# Danh sách hình ảnh chuẩn bị
faces = './faces'
os.listdir(faces)

['HariWon', 'KhoiMy', 'MinhHang', 'NgocTrinh', 'TranThanh', 'TruongGiang']

In [8]:
def display(title, img):
    cv2.imshow(title, img)
    # Chờ một khoảng thời gian
    cv2.waitKey(0)
    # Đóng window
    cv2.destroyAllWindows()

# hiển thị một ảnh
imgTranThanh = './faces/TranThanh/tran_thanh_1.jpg'
img = cv2.imread(imgTranThanh)
display("Ảnh", img)

In [9]:
# Trích xuất đặc trưng cho một ảnh
path = './faces/Tran Thanh/tran thanh .jpg'
img1 = cv2.imread(imgTranThanh)

# Copy cái ảnh
image = img1.copy()

# Lấy chiều cao và chiều rộng của ảnh
h, w = image.shape[:2]

# Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt
imgBlob = cv2.dnn.blobFromImage(image, 1, (300,300), (104, 177, 123), swapRB=False, crop=False)

# Thiết lập đầu vào cho mô hình
detectorModel.setInput(imgBlob)

# Thực hiện việc nhận diện khuôn mặt
detections = detectorModel.forward()

# Kiểm tra xem có khuôn mặt nào hay không?
if (len(detections)>0):
  # Chọn khuôn mặt có độ tin cậy cao nhất (confidence) cao nhất
  i = np.argmax(detections[0, 0, :, 2]) # Giá trị thứ 2 trong 7 giá trị cuối cùng => confidence
  confidence = detections[0, 0, i, 2]

  # Kiểm tra độ tin cậy có lớn hơn 0.5
  if (confidence>0.5):
      # Tính toán hộp bao quanh khuôn mặt
      box = detections[0, 0, i, 3:7]* np.array([w, h, w, h])
      (startX, startY, endX, endY) =box.astype('int')

      # Trích xuất vùng ảnh chứa khuôn mặt ra
      roi = image[startY:endY, startX:endX]
      display("Ảnh", roi)

      # Chuẩn bị dữ liệu đầu vào cho mô hình trích xuất đặc trưng
      faceBlob = cv2.dnn.blobFromImage(roi, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)

      # Thiết lập đầu vào cho mô hình
      descriptorModel.setInput(faceBlob)

      # Thực hiện việc trích xuất đặc trưng
      vectors = descriptorModel.forward()

      # Print vectors
      print(vectors)

[[ 0.09069608  0.09988409 -0.00551813  0.1552697  -0.07495539  0.12252592
   0.0228246  -0.02565777 -0.00257931  0.00994239  0.15322997  0.06305075
   0.04493536 -0.15944254 -0.00212452 -0.02963407  0.03356173  0.14154524
  -0.07185155  0.06261951  0.04588836 -0.01742019  0.02246411  0.08542194
  -0.00591416 -0.16056204 -0.08516108 -0.12267216 -0.13508303  0.10909328
  -0.01610883  0.04325633  0.02116462  0.16204546 -0.0034951  -0.03324474
   0.07955284 -0.03446026 -0.00652454  0.0509337  -0.04054402 -0.08872346
   0.02602424 -0.02719993 -0.06740269 -0.20450138  0.08554076 -0.06307487
  -0.04309677  0.20099887  0.20571528 -0.08715166 -0.12704422  0.08345181
  -0.02951956  0.02949696 -0.03204856 -0.10509726  0.02667746  0.04066859
  -0.04356689 -0.06562763 -0.01894609 -0.2610866   0.0665941  -0.13987043
   0.06094836 -0.15266135 -0.18029132  0.06311119 -0.02938895  0.00923188
  -0.01560777  0.07506275 -0.09644293 -0.07771026  0.08322547  0.00708133
  -0.06887267  0.05027739 -0.08268355 

In [10]:
# Tạo ra một functions để áp dụng cho nhiều ảnh
def myDetect(image_path):
  # Trích xuất đặc trưng cho một ảnh
  img1 = cv2.imread(image_path)

  # Copy cái ảnh
  image = img1.copy()

  # Lấy chiều cao và chiều rộng của ảnh
  h, w = image.shape[:2]

  # Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt
  imgBlob = cv2.dnn.blobFromImage(image, 1, (300,300), (104, 177, 123), swapRB=False, crop=False)

  # Thiết lập đầu vào cho mô hình
  detectorModel.setInput(imgBlob)

  # Thực hiện việc nhận diện khuôn mặt
  detections = detectorModel.forward()

  # Kiểm tra xem có khuôn mặt nào hay không?
  if (len(detections)>0):
    # Chọn khuôn mặt có độ tin cậy cao nhất (confidence) cao nhất
    i = np.argmax(detections[0, 0, :, 2]) # Giá trị thứ 2 trong 7 giá trị cuối cùng => confidence
    confidence = detections[0, 0, i, 2]

    # Kiểm tra độ tin cậy có lớn hơn 0.5
    if (confidence>0.5):
        # Tính toán hộp bao quanh khuôn mặt
        box = detections[0, 0, i, 3:7]* np.array([w, h, w, h])
        (startX, startY, endX, endY) =box.astype('int')

        # Trích xuất vùng ảnh chứa khuôn mặt ra
        roi = image[startY:endY, startX:endX]
        # cv2_imshow(roi)

        # Chuẩn bị dữ liệu đầu vào cho mô hình trích xuất đặc trưng
        faceBlob = cv2.dnn.blobFromImage(roi, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)

        # Thiết lập đầu vào cho mô hình
        descriptorModel.setInput(faceBlob)

        # Thực hiện việc trích xuất đặc trưng
        vectors = descriptorModel.forward()

        # Print vectors
        return vectors

In [11]:
# Apply for all images

# Khởi tạo một dictionary
data = dict(data=[], label=[])

myDir = './faces'
folders = os.listdir(myDir)

# Duyệt qua từng thư mục
for folder in folders:
  path = myDir + "/"+folder
  print(path)

  files =  os.listdir(path)
  for fileName in files:
    try:
        # Gọi hàm trích xuất đặc trưng
        vector = myDetect(path+'/'+fileName)

        # Kiểm tra việc trích xuất đặc trưng thành công:
        if vector is not None:
          #Thêm vector đặc trưng vào trong từ điển với nhãn là tên của thư mục
          data['data'].append(vector)
          data['label'].append(folder)
          print('Trích xuất đặc trưng thành công')
    except:
      # Bỏ qua nếu gặp lỗi
      pass
    

./faces/HariWon
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
./faces/KhoiMy
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuấ

In [15]:
print(data)

{'data': [array([[ 0.08582852,  0.04553367, -0.05129068,  0.13866484, -0.02974133,
         0.14072645,  0.05147205, -0.00713211,  0.01487283,  0.00245537,
         0.14665775,  0.07245764,  0.05608315, -0.15625569, -0.01132801,
         0.00756161, -0.02437488,  0.10802885, -0.12791127,  0.07894436,
         0.06197912, -0.09925503,  0.056269  ,  0.06412957, -0.01221261,
        -0.15472902, -0.10092706, -0.16864763, -0.14531647,  0.05114992,
         0.02231666,  0.11524048, -0.00402289,  0.15895207, -0.06201407,
         0.02125441,  0.02394095, -0.03533982, -0.03060036,  0.04583111,
         0.0368393 , -0.12860501, -0.01272506, -0.02978579, -0.08417507,
        -0.16294576,  0.07397584, -0.04667805, -0.06077962,  0.15938325,
         0.1244565 , -0.11395217, -0.1416257 ,  0.06255874, -0.02041218,
         0.03935378, -0.03277523, -0.05274296,  0.00937332,  0.08167502,
        -0.0700957 , -0.03616229,  0.01417371, -0.25206783,  0.08754769,
        -0.09793797,  0.0137442 , -0.0793

In [12]:
data.keys()

dict_keys(['data', 'label'])

In [13]:
# Tạo một series từ dictionary
labelSeries = pd.Series(data['label'])
# Đếm số lần xuất hiện của mỗi nhãn
labelCounts = labelSeries.value_counts()
print(labelCounts)

TranThanh      23
HariWon        18
KhoiMy         16
TruongGiang    11
MinhHang       10
NgocTrinh       9
Name: count, dtype: int64


In [14]:
# Save data -> file
fileName = './data/data_face_features.pickle'
pickle.dump(data, open(fileName, mode='wb'))